In [1]:
import librosa
import math
import numpy as np
import scipy.signal
from scipy.special import logsumexp
import torch
import torch.nn as nn
import torch.nn.functional as F

class MyNet(nn.Module):
    def __init__(self):
        super(MyNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 5, padding=2)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(32, 64, 5, padding=2)
        self.conv3 = nn.Conv2d(64, 64, 3, padding=1)
        self.conv4 = nn.Conv2d(64, 128, (1, 5))
        self.fc1 = nn.Linear(128, 128)
        self.fc2 = nn.Linear(128, 128)
        self.fc3 = nn.Linear(128, 48)
        self.sm = nn.LogSoftmax(dim=1)

    def forward(self, x):
        x = x.unsqueeze(1)
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = F.relu(self.conv4(x))
        x = x.view(-1, 128)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        x = self.sm(x)
        return x
    
def load_audio_to_melspec_tensor(wavpath, sample_rate=16000):
    window_size = .025
    window_stride = 0.01
    n_dft = 512
    win_length = int(sample_rate * window_size)
    hop_length = int(sample_rate * window_stride)
    y, sr = librosa.load(wavpath, sr=sample_rate)
    y = y - y.mean()
    y = np.append(y[0],y[1:]-.97*y[:-1])
    # compute mel spectrogram
    stft = librosa.stft(y, n_fft=n_dft, hop_length=hop_length,
        win_length=win_length, window=scipy.signal.hamming)
    spec = np.abs(stft)**2
    mel_basis = librosa.filters.mel(sr=sample_rate, n_fft=n_dft, n_mels=40, fmin=20)
    melspec = np.dot(mel_basis, spec)
    logspec = librosa.power_to_db(melspec, ref=np.max)
    logspec = np.transpose(logspec)
    logspec_tensor = torch.tensor(logspec)
    return logspec_tensor

def compute_phone_likelihoods(model, logspec):
    likelihood_list = []
    with torch.no_grad():
        for j in range(6, logspec.size(0) - 5):
            inp = logspec[j-5:j+6,:].unsqueeze(0)
            output = model(inp) # output will be log probabilities over classes
            output = output - math.log(1. / 48) # subtract the logprob of the class priors (assumed to be uniform)
            likelihood_list.append(output[0])
    likelihoods = torch.transpose(torch.stack(likelihood_list, dim=1), 0, 1).numpy()
    return likelihoods

class MyHMM:
    def __init__(self, state_labels, initial_state_distribution, transition_matrix, eps=1e-200):
        self.eps = eps
        self.pi = np.log(initial_state_distribution + eps)
        self.A = np.log(transition_matrix + eps) #A_{ji} is prob of transitioning from state j to state i
        self.labels = state_labels # a list where self.labels[j] is the index of the phone label belonging to the jth state
        print(self.labels)
        self.N_states = len(self.labels)

    def forward(self, state_likelihoods):
        # state_likelihoods.shape is assumed to be (N_timesteps, 48)
        T = state_likelihoods.shape[0]
        
        # Initialization
        log_alpha = self.pi + state_likelihoods[0, self.labels]

        # Induction
        for t in range(1, T):
            log_alpha_new = np.zeros(self.N_states)
            for i in range(self.N_states):
                log_sum = logsumexp(log_alpha + self.A[:, i])
                log_alpha_new[i] = log_sum + state_likelihoods[t, self.labels[i]]
            log_alpha = log_alpha_new
        
        # Termination
        return log_alpha[self.N_states-1]    
 
    def viterbi(self, state_likelihoods):
        T = state_likelihoods.shape[0]

        #initialization
        delta = self.pi + state_likelihoods[0, self.labels]
        psi = np.zeros((T, self.N_states), dtype=int)
        
        #induction
        for t in range(1, T):
            delta_new = np.zeros(self.N_states)
            for j in range(self.N_states):
                temp = delta + self.A[:,j] + state_likelihoods[t,self.labels]
                delta_new[j] = np.max(temp)
                psi[t,j] = np.argmax(temp)
            delta = delta_new
            
        # termination
        q_star_T = np.argmax(delta)

        #backtrace
        q_star = [q_star_T]
        for t in range(T-1, 0, -1):
            q_star_t = psi[t,q_star[-1]]
            q_star.append(q_star_t)
        q_star.reverse()
        return q_star

    
    def viterbi_transition_update(self, state_likelihoods):
        # state_likelihoods.shape is assumed to be (N_timesteps, 48)
        path = self.viterbi(state_likelihoods)
        transitions = np.zeros(self.A.shape)
        for i in range(self.N_states ):
            for j in range(self.N_states ):
                for t in range(len(path) - 1):
                    if (path[t] == i) and (path[t+1] == j):
                        transitions[i][j] += 1
            row_sum = transitions[i].sum()
            transitions[i] /= row_sum
            self.A[i] = np.log(transitions[i] + self.eps)

model = MyNet()
model.load_state_dict(torch.load('lab3_AM.pt'))

lab3_data = np.load('lab3_phone_labels.npz')
phone_labels = list(lab3_data['phone_labels'])
def phones2indices(phones):
    return [phone_labels.index(p) for p in phones]

fee_HMM = MyHMM(phones2indices(['sil', 'f', 'iy', 'sil']), np.array([0.5, 0.5, 0, 0]), np.array([[.9,.1,0,0],[0,.9,.1,0],[0,0,.9,.1],[0,0,0,1]]))
pea_HMM = MyHMM(phones2indices(['sil', 'p', 'iy', 'sil']), np.array([0.5, 0.5, 0, 0]), np.array([[.9,.1,0,0],[0,.9,.1,0],[0,0,.9,.1],[0,0,0,1]]))
rock_HMM = MyHMM(phones2indices(['sil', 'r', 'aa', 'cl', 'k', 'sil']), np.array([0.5,0.5,0,0,0,0]), np.array([[.9,.1,0,0,0,0],[0,.9,.1,0,0,0],[0,0,.9,.1,0,0],[0,0,0,.9,.1,0],[0,0,0,0,.9,.1],[0,0,0,0,0,1]]))
burt_HMM = MyHMM(phones2indices(['sil', 'b', 'er', 'cl', 't', 'sil']), np.array([0.5,0.5,0,0,0,0]), np.array([[.9,.1,0,0,0,0],[0,.9,.1,0,0,0],[0,0,.9,.1,0,0],[0,0,0,.9,.1,0],[0,0,0,0,.9,.1],[0,0,0,0,0,1]]))
see_HMM = MyHMM(phones2indices(['sil', 's', 'iy', 'sil']), np.array([0.5, 0.5, 0, 0]), np.array([[.9,.1,0,0],[0,.9,.1,0],[0,0,.9,.1],[0,0,0,1]]))
she_HMM = MyHMM(phones2indices(['sil', 'sh', 'iy', 'sil']), np.array([0.5, 0.5, 0, 0]), np.array([[.9,.1,0,0],[0,.9,.1,0],[0,0,.9,.1],[0,0,0,1]]))

#ex = np.array([[.9,.1,0,0],[0,.9,.1,0],[0,0,.9,.1],[0,0,0,1]])


# TODO: write your code to use your HMMs below here (or in a new block)
files = ['fee.wav', 'pea.wav', 'rock.wav', 'burt.wav', 'see.wav', 'she.wav']
hmms = [fee_HMM, pea_HMM, rock_HMM, burt_HMM, see_HMM, she_HMM]
words = ['fee', 'pea', 'rock', 'burt', 'see', 'she']

for file in files:
    likelihoods = compute_phone_likelihoods(model, load_audio_to_melspec_tensor(file))
    for i,  hmm in enumerate(hmms):
        print('The log likelihood of', words[i], 'given', file, 'is', hmm.forward(likelihoods))

rock_likelihoods = compute_phone_likelihoods(model, load_audio_to_melspec_tensor('rock.wav'))
rock_optimal_sequence  = rock_HMM.viterbi(rock_likelihoods)
print('The optimal sequence for rock is', rock_optimal_sequence)

log_likelihood = rock_HMM.forward(rock_likelihoods)
print('Log likelihood for rock.wav before update : ', log_likelihood)
original_A = rock_HMM.A.copy()

rock_HMM.viterbi_transition_update(rock_likelihoods)
print('Log likelihood for rock.wav after update : ', rock_HMM.forward(rock_likelihoods))
print("Original transition matrix:")
print(np.exp(original_A))
print("Updated transition matrix: ")
print(np.exp(rock_HMM.A))
print(np.exp(rock_HMM.A) - np.exp(original_A))

[0, 43, 5, 0]
[0, 10, 5, 0]
[0, 4, 17, 9, 26, 0]
[0, 23, 30, 9, 37, 0]
[0, 1, 5, 0]
[0, 14, 5, 0]
The log likelihood of fee given fee.wav is 208.74540805132523
The log likelihood of pea given fee.wav is 179.36370925343732
The log likelihood of rock given fee.wav is -92.06496660504871
The log likelihood of burt given fee.wav is -85.49113294606258
The log likelihood of see given fee.wav is 192.2442375148146
The log likelihood of she given fee.wav is 182.3443420569065
The log likelihood of fee given pea.wav is 250.02150283852316
The log likelihood of pea given pea.wav is 268.16345318587935
The log likelihood of rock given pea.wav is 7.828416239338058
The log likelihood of burt given pea.wav is 71.10708209255513
The log likelihood of see given pea.wav is 236.17288870884235
The log likelihood of she given pea.wav is 234.8391053902661
The log likelihood of fee given rock.wav is -62.592755600530964
The log likelihood of pea given rock.wav is -5.844035664197463
The log likelihood of rock given